In [1]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
from pprint import pprint
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import gc
gc.collect()

0

This notebook is pretty much a fork of the following neat and clean kernel :

https://www.kaggle.com/byfone/neat-baseline-code-for-earthquake-prediction

In [2]:
%%time
# to import the training data efficiently by a chunksize=150000
# why 150000? becaues it is identified that each test file contains 150000 rows of data
# it is also verified that loading the training 'acoustic_data' to int16 
# and 'time_to_failure' to float32 can drastically reduce the memory usage
trainDF = pd.read_csv('train.csv', iterator=True, chunksize=150000, dtype={'acoustic_data':np.int16, 'time_to_failure':np.float32})


Wall time: 34.9 ms


In [3]:
x = next(trainDF)

In [4]:
print(x.describe())
x.max().values[-1]

       acoustic_data  time_to_failure
count  150000.000000    150000.000000
mean        4.884113         1.450068
std         5.101106         0.011248
min       -98.000000         1.430797
25%         3.000000         1.440398
50%         5.000000         1.449999
75%         7.000000         1.459599
max       104.000000         1.469100


1.4691

In [5]:
# create a feature generator
# at the first attempt, let's just use the simple statistic properties of the training time series segment (150000 units)

def feature_generator(x):
    """ 
        Input: Pandas data series, in this case the 'acoustic_data' 
        OUtput: Pandas data series, in this case the features
    """
     
    features = {'mean': x.mean(),
                'std': x.std(),
                'max': x.max(),
                'min': x.min(),
                'q25': x.quantile(.25),
                'q50': x.quantile(.5),
                'q75': x.quantile(.75),
                'iqr': x.quantile(.75) - x.quantile(.25),
                'mad': x.mad(),
                'skew': x.skew(),
                'kurtosis': x.kurtosis()
    }
    
    fseries = pd.Series(features)
    return fseries
    

In [6]:
# create a function to reconstruct the training data and testing data into X_train and y_train format

def xy_train_generator(xy):
    """ 
        input: the chunk of 150000 dataframe
        output: X_train, y_train that is ready for the sklearn machine learning models, where
                X_train is a Data Frame
                y_Train is a Data Series
    """
    
    # get the feature names and drop the first segment (chunksize of 150000 record)
    segment = next(xy)
    cols = feature_generator(segment['acoustic_data']).index
    
    X_train = pd.DataFrame(columns=cols)
    y_train = pd.Series()
    
    for xxyy in tqdm(xy):
        X_train = X_train.append(feature_generator(xxyy['acoustic_data']), ignore_index=True)
        y_train = y_train.append(pd.Series(xxyy['time_to_failure'].values[-1]))
    
    return X_train, y_train

In [7]:
X_train, y_train = xy_train_generator(trainDF)

In [8]:
# let's have a peak of the X_train data
X_train.head()

,mean,std,max,min,q25,q50,q75,iqr,mad,skew,kurtosis
0,4.906393,6.967397,140.0,-106.0,2.0,5.0,7.0,5.0,3.948411,0.217391,33.555211
1,4.902240,6.922305,197.0,-199.0,2.0,5.0,7.0,5.0,3.647117,0.757278,116.548172
2,4.908720,7.301110,145.0,-126.0,2.0,5.0,7.0,5.0,3.826052,0.064531,52.977905
3,4.913513,5.434111,142.0,-144.0,3.0,5.0,7.0,4.0,3.378388,-0.100697,50.215147
4,4.855660,5.687823,120.0,-78.0,2.0,5.0,7.0,5.0,3.560799,0.208810,23.173004


In [9]:
# let's have a peak of the y_train data
y_train.head()

0    1.353196
0    1.313798
0    1.274400
0    1.236097
0    1.196798
dtype: float64

In [10]:
# it is always a good practice to scale the input data, let's use the Standard Scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

In [11]:
# Let's have a look of the after scaling X_train data

X_train_scaled.head()

,mean,std,max,min,q25,q50,q75,iqr,mad,skew,kurtosis
0,1.511941,0.049298,-0.086211,0.162954,-0.405908,0.947251,0.288859,0.474100,0.287099,0.191632,-0.492558
1,1.495717,0.043996,0.122610,-0.187833,-0.405908,0.947251,0.288859,0.474100,0.101267,1.321256,0.684023
2,1.521030,0.088535,-0.067894,0.087516,-0.405908,0.947251,0.288859,0.474100,0.211630,-0.128201,-0.217205
3,1.539754,-0.130984,-0.078884,0.019621,1.594060,0.947251,0.288859,-0.917989,-0.064479,-0.473912,-0.256373
4,1.313763,-0.101153,-0.159482,0.268567,-0.405908,0.947251,0.288859,0.474100,0.048028,0.173679,-0.639746


In [12]:
# now time to get our hands dirty, let's implement a very simple random forest model and check our score

from sklearn.model_selection import train_test_split

XX_train, XX_test, yy_train, yy_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)
print(XX_train.shape, yy_train.shape)
print(XX_test.shape, yy_test.shape)

(3354, 11) (3354,)
(839, 11) (839,)


In [13]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(XX_train, yy_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [14]:
# find the mse when predicting with the test dataset
from sklearn.metrics import mean_absolute_error
yy_pred = rf.predict(XX_test)
MAE = mean_absolute_error(yy_test, yy_pred)
print(MAE)

2.153989798469502


## Parameter tuning

The remainig of this notebook is all about tuning, using random forecast as example
Some reference URLs:
- https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [15]:
# Let's check our current rf parameters
pprint(rf.get_params())

{'bootstrap': True,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 1000,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [16]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [17]:
# Use the random grid to search for best hyperparameters
# First create the base model for hyper parameter tuning
rf_base = RandomForestRegressor()  

# Random search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf_base, param_distributions = random_grid, n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(XX_train, yy_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.9min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 16.6min finished


RandomizedSearchCV(cv=5, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [18]:
# let's see the best parameters from fitting the random search
pprint(rf_random.best_params_)

{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 400}


In [19]:
# also, let's see how does the best rf perform
best_rf = rf_random.best_estimator_
yy_pred = best_rf.predict(XX_test)
MAE = mean_absolute_error(yy_test, yy_pred)
print(MAE)

2.1203438304587183


### Bayesian Optimization with scikit-learn

reference: 
- https://thuijskens.github.io/2016/12/29/bayesian-optimisation/
- https://github.com/thuijskens/bayesian-optimization
- https://github.com/fmfn/BayesianOptimization/blob/master/examples/sklearn_example.py

In [25]:
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization

In [91]:
# define a random forest cross validation function
# should generalize this function to take it most of the sklearn models

def rfc_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf, input_data, target):
    """
        Random Forest cross validation.
        This function will instantiate a random forest classifier with parameters
        interested parameters. By feeding the data and paramters to the model, the resulting targets 
        will be used to perform cross validation. The result of cross validation is then returned.
        
        The goal is to find combinations of interested parameters that minimzes the log loss.
    """
        
    estimator = RandomForestRegressor(
        n_estimators= n_estimators,
        max_features = 'sqrt', 
        max_depth = max_depth, 
        min_samples_split = min_samples_split,
        min_samples_leaf = min_samples_leaf, 
        bootstrap = True, 
        random_state=42,
        n_jobs=-1
    )
    
    cval = cross_val_score(estimator, input_data, target, scoring='neg_mean_absolute_error', cv=3)
    
    return cval.mean()

In [94]:
# now we create a function to tune the random forecst paramters using bayesian optimization

def bOpt_rfc(input_data, target):
    """Apply Bayesian Optimization to Random Forest parameters."""
    def rfc_crossval(n_estimators, max_depth, min_samples_split, min_samples_leaf):
        """Wrapper of RandomForest cross validation.
        Notice how we ensure n_estimators and min_samples_split are casted
        to integer before we pass them along. Moreover, to avoid max_features
        taking values outside the (0, 1) range, we also ensure it is capped
        accordingly.
        """
        r = rfc_cv(n_estimators=int(n_estimators), max_depth = int(max_depth), min_samples_split=int(min_samples_split), \
            min_samples_leaf=int(min_samples_leaf), input_data=input_data, target=target)
        return r
    

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            'n_estimators': (200, 2000),
            'max_depth': (10,110),
            'min_samples_split': (2, 10),
            'min_samples_leaf': (1, 4)
        },
        random_state=42,
        verbose=2
    )
    

    optimizer.maximize(init_points=2, n_iter=50)

    print("Final result:", optimizer.max)

In [95]:
# Let's try it out
bOpt_rfc(XX_train, yy_train)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        | -2.177    |  47.45    |  3.852    |  7.856    |  1.278e+0 |
|  2        | -2.192    |  25.6     |  1.468    |  2.465    |  1.759e+0 |
|  3        | -2.17     |  10.65    |  1.56     |  3.404    |  200.8    |
|  4        | -2.193    |  109.9    |  1.483    |  5.008    |  206.5    |
|  5        | -2.168    |  10.04    |  1.017    |  3.06     |  711.7    |
|  6        | -2.165    |  10.29    |  3.975    |  9.983    |  1.066e+0 |
|  7        | -2.169    |  10.43    |  1.274    |  2.019    |  1.09e+03 |
|  8        | -2.165    |  10.02    |  3.217    |  9.937    |  808.1    |
|  9        | -2.165    |  10.01    |  3.345    |  9.847    |  887.1    |
|  10       | -2.165    |  10.39    |  3.946    |  9.872    |  930.3    |
|  11       | -2.165    |  10.08    |  3.83     |  9.965    |  777.5    |
|  12       | -2.165    |  10.19    | 

In [98]:
# now make a new random forecst regressor using the optimized paramters

rf_bOpt = RandomForestRegressor(n_estimators=1330, bootstrap=True, 
            max_features='sqrt', max_depth=10, min_samples_leaf=4, min_samples_split=9, random_state=42)

rf_bOpt.fit(XX_train, yy_train)
yy_pred = rf_bOpt.predict(XX_test)
MAE = mean_absolute_error(yy_test, yy_pred)
print(MAE)

2.1178132747615064


## Vola, thanks for browsing